In [1]:
import pandas as pd
import time
import logging
from pythonjsonlogger import jsonlogger
from flumine import FlumineSimulation, clients
from collections import OrderedDict, deque, defaultdict
from flumine import BaseStrategy
from flumine.order.trade import Trade
from flumine.order.order import OrderStatus
from flumine.order.ordertype import LimitOrder
from flumine.utils import get_price
from logging_setup import build_logger
import random
import time
from datetime import timedelta

ModuleNotFoundError: No module named 'logging_setup'

In [3]:
class FlumineStrat(BaseStrategy):
    """
    Example strateg
    """
    def __init__(self, enter_threshold, exit_threshold, order_hold, price_add, log_root, log_level, *a, **k):
        self.log = build_logger(log_root,log_level)  # logs/trades.log, rotated nightly
        super().__init__(name="risk_backfave",*a, **k)
        self.hist = defaultdict(lambda: deque(maxlen=400))  # per runner
        self.enter_threshold = enter_threshold
        self.exit_threshold = exit_threshold
        self.order_hold = order_hold
        self.price_add = price_add
        self.startdt = None
        self._last = {}    # order_id -> last size_matched
        self.rows = []     # collected fills: [market_id, selection_id, time, size, price, side, order_id]
        self.pnl = 0.0     


    def add_market(self, market):
        self.log.info("ADD", market.market_id, market.event_name, market.event_type_id)
    
    def check_market_book(self, market, market_book):
        if market_book.status == "OPEN" and market_book.inplay:
            return True
    
    def _price_now(self, r):
        return r.last_price_traded
    
    def _price_n_secs_ago(self, key, now_dt, n=5):
        cutoff = now_dt - timedelta(seconds=n)
        dq = self.hist.get(key)
        if not dq: return None
        for t,p in reversed(dq):
            if t <= cutoff:
                return p
        return None

    def matched_summary(self, r, market):
        back_total = lay_total = 0.0
        back_weighted = lay_weighted = 0.0
    
        for o in market.blotter:
            if o.selection_id != r.selection_id:
                continue
            m = float(getattr(o, "size_matched", 0) or 0)
            if m <= 0:
                continue
            p = float(getattr(o, "average_price_matched", 0) or 0)
            side = str(getattr(o, "side", "")).upper()
            if "BACK" in side:
                back_total += m
                back_weighted += m * p
            elif "LAY" in side:
                lay_total += m
                lay_weighted += m * p
    
        avg_back = back_weighted / back_total if back_total else 0.0
        avg_lay = lay_weighted / lay_total if lay_total else 0.0
        return back_total, avg_back, lay_total, avg_lay

    def best_prices_for_runner(self, r):
        ex = getattr(r, "ex", None)
        atb = getattr(ex, "available_to_back", []) or []
        atl = getattr(ex, "available_to_lay", []) or []
    
        def _extract(item):
            # handle dict or PriceSize
            if isinstance(item, dict):
                return item.get("price"), item.get("size")
            return getattr(item, "price", None), getattr(item, "size", None)
            
        bb_price, bb_size = _extract(atb[0]) if atb else (None, None)
        bl_price, bl_size = _extract(atl[0]) if atl else (None, None)
        return bb_price, bb_size, bl_price, bl_size

    
    def process_market_book(self, market, market_book):
        try:
            if not self.startdt: self.startdt = market_book.publish_time
            elapsed = market_book.publish_time.timestamp() - self.startdt.timestamp()
            # self.log.debug(f"Process_market_book: {market.event_name} {market.market_id}, time elapsed {elapsed},  publishtime:{market_book.publish_time}")
    
            if elapsed > 1:
                for r in market_book.runners:
                    context = f"context : {market.market_id} {r.selection_id}"
                    back_total, avg_back, lay_total, avg_lay = self.matched_summary(r,market)
                    runner_context = self.get_runner_context(market.market_id, r.selection_id, r.handicap)
                    now_dt = market_book.publish_time
                    key = (market.market_id, r.selection_id)
                    p = self._price_now(r)
                    if not p : return
                    self.log.debug(f"Market tick for {context}, price {p}")
                    
                    if p and p < self.enter_threshold:
                        if runner_context.live_trade_count == 0:
                            self.log.info(f"Trigger back trade: {context} placing order, price {p}")
                            back = round(get_price(r.ex.available_to_lay, 0) + self.price_add,2)
                            trade = Trade(market_book.market_id, r.selection_id, r.handicap, self, notes={"entry_px": back})
                            order = trade.create_order(side="BACK", 
                                                       order_type=LimitOrder(back, self.context["stake"])
                                                       )
                            try:
                                market.place_order(order)
                            except Exception as e:
                                self.log.warning(str(e)) 
                            self.log.info({"ORDER PLACED": context, "price":back})
    
                    if p > self.exit_threshold and back_total:
                        bestb, _ , bestl ,_ = self.best_prices_for_runner(r)
                        loss_on_loss_covered_prc = round(lay_total / back_total,2)
                        cover_ratio = 0.3
                        if loss_on_loss_covered_prc < cover_ratio:
                            if runner_context.live_trade_count == 0:
                                self.log.error(f"Seeing reason to hedge: {context}, ltp {p} \
                                \n back_total:{back_total}, avg_back:{avg_back}, lay_total:{lay_total}, avg_lay:{avg_lay}.\
                        \n Loss Cover Ratio:{lay_total}/{back_total} = {loss_on_loss_covered_prc} : Hedging as ratio < {cover_ratio} \
                        \n Bestback : {bestb} , bestlay : {bestl}")
                                self.hedge_selection(r, market, market_book, p, context, size=2, price=bestl)
                                
        except Exception as e:
            self.log.warning(f"Failed to process market book : {str(e)}")
            
    def hedge_selection(self,r, market, market_book, p, context, size, price):
        try:
            self.log.warning(f"LAY order pre send : {r.selection_id}, hedge size {size}@ hedge price {price}")
            trade = Trade(market_book.market_id, r.selection_id, r.handicap, self)
            order = trade.create_order("LAY", order_type=LimitOrder(price, size))
            market.place_order(order)
            self.log.warning(f"LAY order placed : {context} " )
        except Exception as e:
            self.log.warning(f"Failed send LAY order  : {str(e)}")
            

    def process_orders(self, market, orders):
        for order in orders:
            if order.status == OrderStatus.EXECUTABLE:
                if order.elapsed_seconds and order.elapsed_seconds > self.order_hold:
                     market.cancel_order(order)

    def process_closed_market(self, market, market_book):
        self.pnl = 0.0
        self.log.info(f"Processing closed market: {market.event_name}, {market.market_id}")
        for order in market.blotter:
            self.pnl += order.profit
            self.log.warning(f"Order PNL {order.profit}, av size matched: {order.size_matched} av price matched: {order.average_price_matched}, date_time_created: {order.date_time_created}")
        self.log.warning(f"Total pnl for market:{market.event_name}, {market.market_id}, : PNL :: {self.pnl}")
    

In [4]:
markets = pd.read_csv("bought_data_catalogue.csv")["path"].tolist()

In [5]:
def run_once(market, max_order_exposure, max_selection_exposure, stake, enter_threshold, exit_threshold, order_hold, price_add):
    client = clients.SimulatedClient()
    framework = FlumineSimulation(client=client)

    lists =[]
    lists.append(market)
    
    strategy = FlumineStrat(   
        market_filter={"markets": lists},
        max_order_exposure=max_order_exposure,
        max_selection_exposure=max_selection_exposure,
        context={"stake": stake},
        enter_threshold=enter_threshold,
        exit_threshold=exit_threshold,
        order_hold=order_hold,
        price_add=price_add,
        log_root="./logs/backtest/",
        log_level="E")

    framework.add_strategy(strategy)
    framework.run()
    pnl = getattr(strategy, "pnl")
    del framework, client, strategy
    return pnl

In [ ]:
def run_one_trial_drawdown_optim(max_order_exposure, max_selection_exposure, stake, enter_threshold, exit_threshold, order_hold, price_add):    
    pnls = []
    for i in markets:
        pnl = run_once(i,max_order_exposure=max_order_exposure,max_selection_exposure=max_selection_exposure, stake=stake,
                       enter_threshold=enter_threshold, exit_threshold=exit_threshold,  order_hold=order_hold, price_add=price_add)
        pnls.append(pnl)
        print(i, pnl)
    return pnls

def sum_losses(equity):
    result = sum(abs(x) for x in equity if x <= 0)
    return result


pnls = run_one_trial_drawdown_optim(30,30,2,1.2,5.9,17,0.01)
print(sum(pnls), sum_losses(pnls))

 # arams={'max_order_exposure': 30, 'max_selection_exposure': 90, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 5.9, 'order_hold': 17, 'price_add': 0.01}, user_attrs={}, system_attrs={'NSGAIISampler:generation': 2}, intermediate_values={}, distributions={'max_order_exposure': IntDistribution(high=40, log=False, low=5, step=5), 'max_selection_exposure': IntDistribution(high=200, log=False, low=20, step=10), 'stake': FloatDistribution(high=15.0, log=False, low=1.0, step=1.0), 'enter_threshold': FloatDistribution(high=4.0, log=False, low=1.02, step=0.01), 'exit_threshold': FloatDistribution(high=8.0, log=False, low=2.0, step=0.1), 'order_hold': IntDistribution(high=37, log=False, low=1, step=4), 'price_add': FloatDistribution(high=0.08, log=False, low=-0.01, step=0.01)}, trial_id=75, value=None)

hist_data/ADVANCED/2025/Feb/12/33524813/1.232245129 5.34
hist_data/ADVANCED/2025/Feb/12/33475068/1.231541189 0.65
hist_data/ADVANCED/2024/Aug/20/33508295/1.232019003 3.3600000000000008
hist_data/ADVANCED/2024/Aug/20/33504642/1.231954918 0.32
hist_data/ADVANCED/2024/Aug/20/33508296/1.232018919 3.49
hist_data/ADVANCED/2024/Aug/20/33508297/1.232019056 5.0600000000000005
STDOUT: 2025-10-09 18:10:41,466 ERROR ::  Seeing reason to hedge: context : 1.231895741 2120, ltp 6.0                                 
 back_total:2.0, avg_back:1.17, lay_total:0.0, avg_lay:0.0.                        
 Loss Cover Ratio:0.0/2.0 = 0.0 : Hedging as ratio < 0.3                         
 Bestback : 5.5 , bestlay : 6
hist_data/ADVANCED/2024/Aug/18/33501074/1.231895741 3.77
hist_data/ADVANCED/2024/Aug/18/33501073/1.231895769 3.750000000000001
hist_data/ADVANCED/2024/Aug/18/33504088/1.231946357 1.6400000000000001
STDOUT: 2025-10-09 18:10:55,151 ERROR ::  Seeing reason to hedge: context : 1.231887491 10787826, ltp

In [ ]:
import optuna
from optuna.samplers import NSGAIISampler
from optuna.pruners import SuccessiveHalvingPruner


def objective(trial):
    max_order_exposure = trial.suggest_int("max_order_exposure", 5, 40, step=5)
    max_selection_exposure = trial.suggest_int("max_selection_exposure", 20, 200, step=10)
    stake = trial.suggest_float("stake", 1.0, 15.0, step=1)
    enter_threshold = trial.suggest_float("enter_threshold", 1.02, 4, step=0.01)
    exit_threshold = trial.suggest_float("exit_threshold", 2, 8, step=0.1)
    order_hold = trial.suggest_int("order_hold", 1, 37, step=4)
    price_add = trial.suggest_float("price_add", -0.01, 0.08 , step=0.01)

    equity = run_one_trial_drawdown_optim(
        max_order_exposure=max_order_exposure,
        max_selection_exposure=max_selection_exposure,
        stake=stake,
        enter_threshold=enter_threshold,
        exit_threshold=exit_threshold,
        order_hold=order_hold,
        price_add=price_add
    )
    
    pnl_sum = sum(equity)

    sum_loss = sum_losses(equity)
    print("pnls: ", equity, "\nsum:", pnl_sum, "sum losses : -", sum_loss)
    return pnl_sum, sum_loss

study = optuna.create_study(directions=["maximize", "minimize"],sampler=NSGAIISampler(population_size=28))
study.optimize(objective, n_trials=300, n_jobs=1)

# Choose a Pareto-optimal trial (e.g., lowest MDD above a PnL floor)
best = min(study.best_trials, key=lambda t: (t.values[1], -t.values[0]))
print("Best (PnL, MDD):", best.values)
print("Params:", best.params)


In [ ]:
# Trial 82 f [59.2, 83.27000000000001] and parameters: {'max_order_exposure': 25, 'max_selection_exposure': 20, 'stake': 7.0, 'enter_threshold': 3.14, 'exit_threshold': 4.0, 'order_hold': 9, 'price_add': 0.0}.
# Trial 81 f [81.99000000000001, 14.479999999999997] and parameters: {'max_order_exposure': 15, 'max_selection_exposure': 30, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 3.1, 'order_hold': 13, 'price_add': 0.01}.
# Trial 75 f [87.41000000000003, 7.68] and parameters: {'max_order_exposure': 30, 'max_selection_exposure': 90, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 5.9, 'order_hold': 17, 'price_add': 0.01}.
# Trial 65 f [53.540000000000006, 21.71] and parameters: {'max_order_exposure': 30, 'max_selection_exposure': 30, 'stake': 2.0, 'enter_threshold': 1.2, 'exit_threshold': 5.9, 'order_hold': 13, 'price_add': 0.01}.
# Trial 56 f [57.92, 73.95] and parameters: {'max_order_exposure': 15, 'max_selection_exposure': 20, 'stake': 1.0, 'enter_threshold': 1.5, 'exit_threshold': 3.8, 'order_hold': 5, 'price_add': 0.01}.
# Trial 53 f [92.24, 54.10000000000001] and parameters: {'max_order_exposure': 25, 'max_selection_exposure': 20, 'stake': 7.0, 'enter_threshold': 3.48, 'exit_threshold': 8.0, 'order_hold': 9, 'price_add': -0.01}.
# Trial 43 f [682.89, 516.9399999999999] and parameters: {'max_order_exposure': 10, 'max_selection_exposure': 160, 'stake': 6.0, 'enter_threshold': 3.95, 'exit_threshold': 5.0, 'order_hold': 17, 'price_add': 0.049999999999999996}.


In [ ]:
# Choose a Pareto-optimal trial (e.g., lowest MDD above a PnL floor)
best = min(study.best_trials, key=lambda t: (t.values[1], -t.values[0]))
print("Best (PnL, MDD):", best.values)
print("Params:", best.params)


In [ ]:
lister = [ i for i in study.best_trials if i.values[0] != 0.0]
for i in lister:
    print("\n", i)